# W5 Practice - 220929

In [6]:
import pandas as pd

long_df = pd.read_csv(
    './data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

datatype       date  temp_C  temp_F
0     TMAX 2018-10-01    21.1   69.98
1     TMIN 2018-10-01     8.9   48.02
2     TOBS 2018-10-01    13.9   57.02
3     TMAX 2018-10-02    23.9   75.02
4     TMIN 2018-10-02    13.9   57.02

## Pivoting

In [8]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

datatype    TMAX  TMIN  TOBS
date                        
2018-10-01  21.1   8.9  13.9
2018-10-02  23.9  13.9  17.2
2018-10-03  25.0  15.6  16.1
2018-10-04  22.8  11.7  11.7
2018-10-05  23.3  11.7  18.9

In [9]:
pivoted_df.describe()

datatype       TMAX       TMIN       TOBS
count     31.000000  31.000000  31.000000
mean      16.829032   7.561290  10.022581
std        5.714962   6.513252   6.596550
min        7.800000  -1.100000  -1.100000
25%       12.750000   2.500000   5.550000
50%       16.100000   6.700000   8.300000
75%       21.950000  13.600000  16.100000
max       26.700000  17.800000  21.700000

In [10]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

In [11]:
pivoted_df['temp_F']['TMIN'].head()

date
2018-10-01    48.02
2018-10-02    57.02
2018-10-03    60.08
2018-10-04    53.06
2018-10-05    53.06
Name: TMIN, dtype: float64

## Unstack

In [12]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', 'datatype'])

In [13]:
multi_index_df.head()

temp_C  temp_F
date       datatype                
2018-10-01 TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02

In [14]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

In [15]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

<ipython-input-15-515a4e4ec9fd>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  extra_data = long_df.append([{


temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96

In [16]:
extra_data.unstack().head()

temp_C                   temp_F                     
datatype     TAVG  TMAX  TMIN  TOBS   TAVG   TMAX   TMIN   TOBS
date                                                           
2018-10-01   10.0  21.1   8.9  13.9   50.0  69.98  48.02  57.02
2018-10-02    NaN  23.9  13.9  17.2    NaN  75.02  57.02  62.96
2018-10-03    NaN  25.0  15.6  16.1    NaN  77.00  60.08  60.98
2018-10-04    NaN  22.8  11.7  11.7    NaN  73.04  53.06  53.06
2018-10-05    NaN  23.3  11.7  18.9    NaN  73.94  53.06  66.02

In [17]:
extra_data.unstack(fill_value=-40).head()

temp_C                   temp_F                     
datatype     TAVG  TMAX  TMIN  TOBS   TAVG   TMAX   TMIN   TOBS
date                                                           
2018-10-01   10.0  21.1   8.9  13.9   50.0  69.98  48.02  57.02
2018-10-02  -40.0  23.9  13.9  17.2  -40.0  75.02  57.02  62.96
2018-10-03  -40.0  25.0  15.6  16.1  -40.0  77.00  60.08  60.98
2018-10-04  -40.0  22.8  11.7  11.7  -40.0  73.04  53.06  53.06
2018-10-05  -40.0  23.3  11.7  18.9  -40.0  73.94  53.06  66.02

## Melting

In [18]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

date  TMAX  TMIN  TOBS
0  2018-10-01  21.1   8.9  13.9
1  2018-10-02  23.9  13.9  17.2
2  2018-10-03  25.0  15.6  16.1
3  2018-10-04  22.8  11.7  11.7
4  2018-10-05  23.3  11.7  18.9

In [19]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

date measurement  temp_C
0  2018-10-01        TMAX    21.1
1  2018-10-02        TMAX    23.9
2  2018-10-03        TMAX    25.0
3  2018-10-04        TMAX    22.8
4  2018-10-05        TMAX    23.3

## Stack

In [20]:
wide_df.set_index('date', inplace=True)
wide_df.head()

TMAX  TMIN  TOBS
date                        
2018-10-01  21.1   8.9  13.9
2018-10-02  23.9  13.9  17.2
2018-10-03  25.0  15.6  16.1
2018-10-04  22.8  11.7  11.7
2018-10-05  23.3  11.7  18.9

In [21]:
stacked_series = wide_df.stack()
stacked_series.head()

date            
2018-10-01  TMAX    21.1
            TMIN     8.9
            TOBS    13.9
2018-10-02  TMAX    23.9
            TMIN    13.9
dtype: float64

In [22]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

values
date                   
2018-10-01 TMAX    21.1
           TMIN     8.9
           TOBS    13.9
2018-10-02 TMAX    23.9
           TMIN    13.9

In [23]:
stacked_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', None])

In [24]:
stacked_df.index.names

FrozenList(['date', None])

In [25]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

FrozenList(['date', 'datatype'])

## Finding problematic data

In [26]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

In [27]:
df.head()

date            station  PRCP  SNOW  SNWD    TMAX  TMIN  \
0  2018-01-01T00:00:00                  ?   0.0   0.0  -inf  5505.0 -40.0   
1  2018-01-01T00:00:00                  ?   0.0   0.0  -inf  5505.0 -40.0   
2  2018-01-01T00:00:00                  ?   0.0   0.0  -inf  5505.0 -40.0   
3  2018-01-02T00:00:00  GHCND:USC00280907   0.0   0.0  -inf    -8.3 -16.1   
4  2018-01-03T00:00:00  GHCND:USC00280907   0.0   0.0  -inf    -4.4 -13.9   

   TOBS  WESF inclement_weather  
0   NaN   NaN               NaN  
1   NaN   NaN               NaN  
2   NaN   NaN               NaN  
3 -12.2   NaN             False  
4 -13.3   NaN             False

In [28]:
df.describe()

/Users/ojoo/opt/anaconda3/envs/venv/lib/python3.8/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


PRCP        SNOW   SNWD         TMAX        TMIN        TOBS  \
count  765.000000  577.000000  577.0   765.000000  765.000000  398.000000   
mean     5.360392    4.202773    NaN  2649.175294  -15.914379    8.632161   
std     10.002138   25.086077    NaN  2744.156281   24.242849    9.815054   
min      0.000000    0.000000   -inf   -11.700000  -40.000000  -16.100000   
25%      0.000000    0.000000    NaN    13.300000  -40.000000    0.150000   
50%      0.000000    0.000000    NaN    32.800000  -11.100000    8.300000   
75%      5.800000    0.000000    NaN  5505.000000    6.700000   18.300000   
max     61.700000  229.000000    inf  5505.000000   23.900000   26.100000   

            WESF  
count  11.000000  
mean   16.290909  
std     9.489832  
min     1.800000  
25%     8.600000  
50%    19.300000  
75%    24.900000  
max    28.700000

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


In [30]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0]

765

In [31]:
contain_nulls.head(10)

date            station  PRCP   SNOW  SNWD    TMAX  TMIN  \
0  2018-01-01T00:00:00                  ?   0.0    0.0  -inf  5505.0 -40.0   
1  2018-01-01T00:00:00                  ?   0.0    0.0  -inf  5505.0 -40.0   
2  2018-01-01T00:00:00                  ?   0.0    0.0  -inf  5505.0 -40.0   
3  2018-01-02T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -8.3 -16.1   
4  2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -4.4 -13.9   
5  2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -4.4 -13.9   
6  2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -4.4 -13.9   
7  2018-01-04T00:00:00                  ?  20.6  229.0   inf  5505.0 -40.0   
8  2018-01-04T00:00:00                  ?  20.6  229.0   inf  5505.0 -40.0   
9  2018-01-05T00:00:00                  ?   0.3    NaN   NaN  5505.0 -40.0   

   TOBS  WESF inclement_weather  
0   NaN   NaN               NaN  
1   NaN   NaN               NaN  
2   NaN   NaN               NaN  
3 -12.2   NaN             False  
4 -13.3   NaN             False  
5 -13.3   NaN             False  
6 -13.3   NaN             False  
7   NaN  19.3              True  
8   NaN  19.3              True  
9   NaN   NaN               NaN

In [32]:
df[df.inclement_weather == 'NaN'].shape[0] # X

0

In [33]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0] # X

0

In [34]:
df[df.inclement_weather.isna()].shape[0]

357

In [35]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

577

In [36]:
def get_inf_count(df):
    """Find the number of inf/-inf values per column in the dataframe"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

In [37]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

count        mean        std   min   25%    50%    75%  \
np.inf Snow Depth    24.0  101.041667  74.498018  13.0  25.0  120.5  152.0   
-np.inf Snow Depth  553.0    0.000000   0.000000   0.0   0.0    0.0    0.0   

                      max  
np.inf Snow Depth   229.0  
-np.inf Snow Depth    0.0

In [38]:
df.describe(include='object')

date            station inclement_weather
count                   765                765               408
unique                  324                  2                 2
top     2018-07-05T00:00:00  GHCND:USC00280907             False
freq                      8                398               384

In [39]:
df[df.duplicated()].shape[0]

284

In [40]:
df[df.duplicated(keep=False)].shape[0]

482

In [41]:
df[df.duplicated(['date', 'station'])].shape[0]

284

In [42]:
df[df.duplicated()].head()

date            station  PRCP   SNOW  SNWD    TMAX  TMIN  \
1  2018-01-01T00:00:00                  ?   0.0    0.0  -inf  5505.0 -40.0   
2  2018-01-01T00:00:00                  ?   0.0    0.0  -inf  5505.0 -40.0   
5  2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -4.4 -13.9   
6  2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0  -inf    -4.4 -13.9   
8  2018-01-04T00:00:00                  ?  20.6  229.0   inf  5505.0 -40.0   

   TOBS  WESF inclement_weather  
1   NaN   NaN               NaN  
2   NaN   NaN               NaN  
5 -13.3   NaN             False  
6 -13.3   NaN             False  
8   NaN  19.3              True

## Mitigating Issues

In [43]:
df[df.WESF.notna()].station.unique()

array(['?'], dtype=object)

In [44]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence 
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.shape

(324, 8)

In [45]:
df_deduped.head()

PRCP   SNOW  SNWD    TMAX  TMIN  TOBS  WESF inclement_weather
date                                                                     
2018-01-01   0.0    0.0  -inf  5505.0 -40.0   NaN   NaN               NaN
2018-01-02   0.0    0.0  -inf    -8.3 -16.1 -12.2   NaN             False
2018-01-03   0.0    0.0  -inf    -4.4 -13.9 -13.3   NaN             False
2018-01-04  20.6  229.0   inf  5505.0 -40.0   NaN  19.3              True
2018-01-05  14.2  127.0   inf    -4.4 -13.9 -13.9   NaN              True

In [46]:
df_deduped.dropna().shape

(4, 8)

In [47]:
df_deduped.dropna(how='all').shape

(324, 8)

In [48]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

(293, 8)

In [49]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

In [50]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

PRCP   SNOW  SNWD    TMAX  TMIN  TOBS  WESF inclement_weather
date                                                                     
2018-01-01   0.0    0.0  -inf  5505.0 -40.0   NaN   0.0               NaN
2018-01-02   0.0    0.0  -inf    -8.3 -16.1 -12.2   0.0             False
2018-01-03   0.0    0.0  -inf    -4.4 -13.9 -13.3   0.0             False
2018-01-04  20.6  229.0   inf  5505.0 -40.0   NaN  19.3              True
2018-01-05  14.2  127.0   inf    -4.4 -13.9 -13.9   0.0              True

In [51]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

In [52]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill')
).head()

PRCP   SNOW  SNWD  TMAX  TMIN  TOBS  WESF inclement_weather
date                                                                   
2018-01-01   0.0    0.0  -inf   NaN   NaN   NaN   0.0               NaN
2018-01-02   0.0    0.0  -inf  -8.3 -16.1 -12.2   0.0             False
2018-01-03   0.0    0.0  -inf  -4.4 -13.9 -13.3   0.0             False
2018-01-04  20.6  229.0   inf  -4.4 -13.9   NaN  19.3              True
2018-01-05  14.2  127.0   inf  -4.4 -13.9 -13.9   0.0              True

In [53]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

PRCP   SNOW           SNWD  TMAX  TMIN  TOBS  WESF  \
date                                                             
2018-01-01   0.0    0.0 -1.797693e+308   NaN   NaN   NaN   0.0   
2018-01-02   0.0    0.0 -1.797693e+308  -8.3 -16.1 -12.2   0.0   
2018-01-03   0.0    0.0 -1.797693e+308  -4.4 -13.9 -13.3   0.0   
2018-01-04  20.6  229.0  1.797693e+308   NaN   NaN   NaN  19.3   
2018-01-05  14.2  127.0  1.797693e+308  -4.4 -13.9 -13.9   0.0   

           inclement_weather  
date                          
2018-01-01               NaN  
2018-01-02             False  
2018-01-03             False  
2018-01-04              True  
2018-01-05              True

In [54]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

PRCP   SNOW   SNWD  TMAX  TMIN  TOBS  WESF inclement_weather
date                                                                    
2018-01-01   0.0    0.0    0.0   NaN   NaN   NaN   0.0               NaN
2018-01-02   0.0    0.0    0.0  -8.3 -16.1 -12.2   0.0             False
2018-01-03   0.0    0.0    0.0  -4.4 -13.9 -13.3   0.0             False
2018-01-04  20.6  229.0  229.0   NaN   NaN   NaN  19.3              True
2018-01-05  14.2  127.0  127.0  -4.4 -13.9 -13.9   0.0              True

In [55]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

PRCP   SNOW  SNWD  TMAX  TMIN  TOBS  WESF inclement_weather
date                                                                   
2018-01-01   0.0    0.0  -inf  14.4   5.6  10.0   0.0               NaN
2018-01-02   0.0    0.0  -inf  -8.3 -16.1 -12.2   0.0             False
2018-01-03   0.0    0.0  -inf  -4.4 -13.9 -13.3   0.0             False
2018-01-04  20.6  229.0   inf  14.4   5.6  10.0  19.3              True
2018-01-05  14.2  127.0   inf  -4.4 -13.9 -13.9   0.0              True

In [56]:
df_deduped.apply(
    # rolling calculations will be covered in chapter 4, this is a rolling 7-day median
    # we set min_periods (# of periods required for calculation) to 0 so we always get a result 
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

PRCP   SNOW  SNWD   TMAX  TMIN   TOBS  WESF inclement_weather
date                                                                     
2018-01-01   0.0    0.0  -inf    NaN   NaN    NaN   0.0               NaN
2018-01-02   0.0    0.0  -inf  -8.30 -16.1 -12.20   0.0             False
2018-01-03   0.0    0.0  -inf  -4.40 -13.9 -13.30   0.0             False
2018-01-04  20.6  229.0   inf  -6.35 -15.0 -12.75  19.3              True
2018-01-05  14.2  127.0   inf  -4.40 -13.9 -13.90   0.0              True
2018-01-06   0.0    0.0  -inf -10.00 -15.6 -15.00   0.0             False
2018-01-07   0.0    0.0  -inf -11.70 -17.2 -16.10   0.0             False
2018-01-08   0.0    0.0  -inf  -7.80 -16.7  -8.30   0.0             False
2018-01-10   0.0    0.0  -inf   5.00  -7.8  -7.80   0.0             False
2018-01-11   0.0    0.0  -inf   4.40  -7.8   1.10   0.0             False

In [57]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

PRCP   SNOW  SNWD  TMAX   TMIN   TOBS  WESF inclement_weather
2018-01-01   0.0    0.0  -inf   NaN    NaN    NaN   0.0               NaN
2018-01-02   0.0    0.0  -inf  -8.3 -16.10 -12.20   0.0             False
2018-01-03   0.0    0.0  -inf  -4.4 -13.90 -13.30   0.0             False
2018-01-04  20.6  229.0   inf  -4.4 -13.90 -13.60  19.3              True
2018-01-05  14.2  127.0   inf  -4.4 -13.90 -13.90   0.0              True
2018-01-06   0.0    0.0  -inf -10.0 -15.60 -15.00   0.0             False
2018-01-07   0.0    0.0  -inf -11.7 -17.20 -16.10   0.0             False
2018-01-08   0.0    0.0  -inf  -7.8 -16.70  -8.30   0.0             False
2018-01-09   0.0    0.0  -inf  -1.4 -12.25  -8.05   0.0               NaN
2018-01-10   0.0    0.0  -inf   5.0  -7.80  -7.80   0.0             False

## Filtering data

In [61]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

date datatype            station attributes  value
0  2018-01-01T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0
1  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0015       ,,N,    0.0
2  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0015       ,,N,    0.0
3  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0017       ,,N,    0.0
4  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0017       ,,N,    0.0

### Find all entries where datatype is “PRCP“ and station ID containing “S0007”

In [63]:
weather[
    (weather.datatype == 'PRCP')
    & weather.station.str.contains('S0007')
]

date datatype            station attributes  value
79     2018-01-01T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.0
311    2018-01-02T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.0
545    2018-01-03T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.0
788    2018-01-04T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    4.1
1044   2018-01-05T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,   18.0
...                    ...      ...                ...        ...    ...
77762  2018-12-27T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.0
77974  2018-12-28T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    4.3
78191  2018-12-29T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,   25.9
78408  2018-12-30T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.3
78641  2018-12-31T00:00:00     PRCP  GHCND:US1NYNS0007       ,,N,    0.0

[349 rows x 5 columns]

### How many unique values in “attributes” variable?

In [68]:
weather.attributes.unique().shape

(26,)

### What is the most frequent one?

In [98]:
weather.attributes.describe() # ,,N,  # value_counts()

count     78780
unique       26
top        ,,N,
freq      34153
Name: attributes, dtype: object

## Merging DataFrames

In [72]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

id                           name   latitude  longitude  \
0  GHCND:US1CTFR0022        STAMFORD 2.6 SSW, CT US  41.064100 -73.577000   
1  GHCND:US1CTFR0039          STAMFORD 4.2 S, CT US  41.037788 -73.568176   
2  GHCND:US1NJBG0001      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983   
3  GHCND:US1NJBG0002  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358   
4  GHCND:US1NJBG0003           TENAFLY 1.3 W, NJ US  40.914670 -73.977500   

   elevation  
0       36.6  
1        6.4  
2       20.1  
3       16.8  
4       21.6

In [73]:
weather.head()

date datatype            station attributes  value
0  2018-01-01T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0
1  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0015       ,,N,    0.0
2  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0015       ,,N,    0.0
3  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0017       ,,N,    0.0
4  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0017       ,,N,    0.0

In [74]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

In [75]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

In [76]:
station_info.shape[0], weather.shape[0]

(279, 78780)

In [77]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

In [78]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

date datatype            station attributes  value  \
10739  2018-08-07T00:00:00     SNOW  GHCND:US1NJMN0069       ,,N,    0.0   
45188  2018-12-21T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400   16.7   
59823  2018-01-15T00:00:00     WDF5  GHCND:USW00094741       ,,W,   40.0   
10852  2018-10-31T00:00:00     PRCP  GHCND:US1NJMN0069      T,,N,    0.0   
46755  2018-05-05T00:00:00     SNOW  GHCND:USW00014734       ,,W,    0.0   

                      id                                         name  \
10739  GHCND:US1NJMN0069                   LONG BRANCH 1.7 SSW, NJ US   
45188  GHCND:USW00014732                     LAGUARDIA AIRPORT, NY US   
59823  GHCND:USW00094741                     TETERBORO AIRPORT, NJ US   
10852  GHCND:US1NJMN0069                   LONG BRANCH 1.7 SSW, NJ US   
46755  GHCND:USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US   

        latitude  longitude  elevation  
10739  40.275368 -74.006027        9.4  
45188  40.779440 -73.880350        3.4  
59823  40.850000 -74.061390        2.7  
10852  40.275368 -74.006027        9.4  
46755  40.682500 -74.169400        2.1

In [79]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

date datatype            station attributes  value  \
10739  2018-08-07T00:00:00     SNOW  GHCND:US1NJMN0069       ,,N,    0.0   
45188  2018-12-21T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400   16.7   
59823  2018-01-15T00:00:00     WDF5  GHCND:USW00094741       ,,W,   40.0   
10852  2018-10-31T00:00:00     PRCP  GHCND:US1NJMN0069      T,,N,    0.0   
46755  2018-05-05T00:00:00     SNOW  GHCND:USW00014734       ,,W,    0.0   

                                              name   latitude  longitude  \
10739                   LONG BRANCH 1.7 SSW, NJ US  40.275368 -74.006027   
45188                     LAGUARDIA AIRPORT, NY US  40.779440 -73.880350   
59823                     TETERBORO AIRPORT, NJ US  40.850000 -74.061390   
10852                   LONG BRANCH 1.7 SSW, NJ US  40.275368 -74.006027   
46755  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  40.682500 -74.169400   

       elevation  
10739        9.4  
45188        3.4  
59823        2.7  
10852        9.4  
46755        2.1

In [80]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

date datatype station attributes  value                 id  \
0    NaN      NaN     NaN        NaN    NaN  GHCND:US1CTFR0022   
344  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0001   
345  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0002   
718  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0005   
719  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0006   

                              name   latitude  longitude  elevation  
0          STAMFORD 2.6 SSW, CT US  41.064100 -73.577000       36.6  
344      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983       20.1  
345  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358       16.8  
718        WESTWOOD 0.8 ESE, NJ US  40.983041 -74.015858       15.8  
719            RAMSEY 0.6 E, NJ US  41.058611 -74.134068      112.2

In [81]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

True

In [82]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

In [83]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

date datatype            station attributes  value  \
23634  2018-04-12T00:00:00     PRCP  GHCND:US1NYNS0043       ,,N,    0.0   
25742  2018-03-25T00:00:00     PRCP  GHCND:US1NYSF0061       ,,N,    0.0   
60645  2018-04-16T00:00:00     TMIN  GHCND:USW00094741       ,,W,    3.9   
70764  2018-03-23T00:00:00     SNWD  GHCND:US1NJHD0002       ,,N,  203.0   
78790                  NaN      NaN                NaN        NaN    NaN   
78800                  NaN      NaN                NaN        NaN    NaN   

                      id                         name   latitude  longitude  \
23634  GHCND:US1NYNS0043     PLAINVIEW 0.4 ENE, NY US  40.785919 -73.466873   
25742  GHCND:US1NYSF0061     CENTERPORT 0.9 SW, NY US  40.891689 -73.383133   
60645                NaN                          NaN        NaN        NaN   
70764                NaN                          NaN        NaN        NaN   
78790  GHCND:US1NYQN0033  HOWARD BEACH 0.4 NNW, NY US  40.662099 -73.841345   
78800  GHCND:US1NYWC0009    NEW ROCHELLE 1.3 S, NY US  40.904000 -73.777000   

       elevation      _merge  
23634       56.7        both  
25742       53.6        both  
60645        NaN   left_only  
70764        NaN   left_only  
78790        2.1  right_only  
78800       21.9  right_only

In [84]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

date datatype            station attributes  value  \
23634  2018-04-12T00:00:00     PRCP  GHCND:US1NYNS0043       ,,N,    0.0   
25742  2018-03-25T00:00:00     PRCP  GHCND:US1NYSF0061       ,,N,    0.0   
60645  2018-04-16T00:00:00     TMIN  GHCND:USW00094741       ,,W,    3.9   
70764  2018-03-23T00:00:00     SNWD  GHCND:US1NJHD0002       ,,N,  203.0   
78790                  NaN      NaN                NaN        NaN    NaN   
78800                  NaN      NaN                NaN        NaN    NaN   

                      id                         name   latitude  longitude  \
23634  GHCND:US1NYNS0043     PLAINVIEW 0.4 ENE, NY US  40.785919 -73.466873   
25742  GHCND:US1NYSF0061     CENTERPORT 0.9 SW, NY US  40.891689 -73.383133   
60645                NaN                          NaN        NaN        NaN   
70764                NaN                          NaN        NaN        NaN   
78790  GHCND:US1NYQN0033  HOWARD BEACH 0.4 NNW, NY US  40.662099 -73.841345   
78800  GHCND:US1NYWC0009    NEW ROCHELLE 1.3 S, NY US  40.904000 -73.777000   

       elevation      _merge  
23634       56.7        both  
25742       53.6        both  
60645        NaN   left_only  
70764        NaN   left_only  
78790        2.1  right_only  
78800       21.9  right_only

In [85]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

station  PRCP   SNOW    TMAX  TMIN  TOBS  WESF  \
date                                                                            
2018-01-01T00:00:00                  ?   0.0    0.0  5505.0 -40.0   NaN   NaN   
2018-01-02T00:00:00  GHCND:USC00280907   0.0    0.0    -8.3 -16.1 -12.2   NaN   
2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0    -4.4 -13.9 -13.3   NaN   
2018-01-04T00:00:00                  ?  20.6  229.0  5505.0 -40.0   NaN  19.3   
2018-01-05T00:00:00                  ?   0.3    NaN  5505.0 -40.0   NaN   NaN   

                    inclement_weather  
date                                   
2018-01-01T00:00:00               NaN  
2018-01-02T00:00:00             False  
2018-01-03T00:00:00             False  
2018-01-04T00:00:00              True  
2018-01-05T00:00:00               NaN

In [86]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

In [87]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

PRCP_x  SNOW_x  TMAX  TMIN  TOBS inclement_weather_x  \
date                                                                        
2018-01-30T00:00:00     0.0     0.0   6.7  -1.7  -0.6               False   
2018-03-08T00:00:00    48.8     NaN   1.1  -0.6   1.1               False   
2018-03-13T00:00:00     4.1    51.0   5.6  -3.9   0.0                True   
2018-03-21T00:00:00     0.0     0.0   2.8  -2.8   0.6               False   
2018-04-02T00:00:00     9.1   127.0  12.8  -1.1  -1.1                True   

                     PRCP_y  SNOW_y  WESF inclement_weather_y  
date                                                           
2018-01-30T00:00:00     1.5    13.0   1.8                True  
2018-03-08T00:00:00    28.4     NaN  28.7                 NaN  
2018-03-13T00:00:00     3.0    13.0   3.0                True  
2018-03-21T00:00:00     6.6   114.0   8.6                True  
2018-04-02T00:00:00    14.0   152.0  15.2                True

In [88]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

PRCP   SNOW  TMAX  TMIN  TOBS inclement_weather  PRCP_?  \
date                                                                           
2018-01-30T00:00:00   0.0    0.0   6.7  -1.7  -0.6             False     1.5   
2018-03-08T00:00:00  48.8    NaN   1.1  -0.6   1.1             False    28.4   
2018-03-13T00:00:00   4.1   51.0   5.6  -3.9   0.0              True     3.0   
2018-03-21T00:00:00   0.0    0.0   2.8  -2.8   0.6             False     6.6   
2018-04-02T00:00:00   9.1  127.0  12.8  -1.1  -1.1              True    14.0   

                     SNOW_?  WESF inclement_weather_?  
date                                                   
2018-01-30T00:00:00    13.0   1.8                True  
2018-03-08T00:00:00     NaN  28.7                 NaN  
2018-03-13T00:00:00    13.0   3.0                True  
2018-03-21T00:00:00   114.0   8.6                True  
2018-04-02T00:00:00   152.0  15.2                True

In [89]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

PRCP   SNOW  TMAX  TMIN  TOBS inclement_weather  PRCP_?  \
date                                                                           
2018-01-30T00:00:00   0.0    0.0   6.7  -1.7  -0.6             False     1.5   
2018-03-08T00:00:00  48.8    NaN   1.1  -0.6   1.1             False    28.4   
2018-03-13T00:00:00   4.1   51.0   5.6  -3.9   0.0              True     3.0   
2018-03-21T00:00:00   0.0    0.0   2.8  -2.8   0.6             False     6.6   
2018-04-02T00:00:00   9.1  127.0  12.8  -1.1  -1.1              True    14.0   

                     SNOW_?  WESF inclement_weather_?  
date                                                   
2018-01-30T00:00:00    13.0   1.8                True  
2018-03-08T00:00:00     NaN  28.7                 NaN  
2018-03-13T00:00:00    13.0   3.0                True  
2018-03-21T00:00:00   114.0   8.6                True  
2018-04-02T00:00:00   152.0  15.2                True

In [90]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

In [91]:
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

In [92]:
weather.index.difference(station_info.index)

Index([], dtype='object')

In [93]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

In [94]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

True

In [95]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

In [96]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

True